In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [1]:
import numpy as np # linear algebra
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd 
import nltk                                # Python library for NLP
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer ,WordNetLemmatizer        # module for stemming and lemmatizer
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings
import re 
import string


In [2]:
train_data = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_data = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train_data


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [5]:
train_data_cleaned=train_data.drop(columns=['keyword','location','id'])
train_data_cleaned.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_data_cleaned["text"]

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [7]:
train_data_cleaned.info

<bound method DataFrame.info of                                                    text  target
0     Our Deeds are the Reason of this #earthquake M...       1
1                Forest fire near La Ronge Sask. Canada       1
2     All residents asked to 'shelter in place' are ...       1
3     13,000 people receive #wildfires evacuation or...       1
4     Just got sent this photo from Ruby #Alaska as ...       1
...                                                 ...     ...
7608  Two giant cranes holding a bridge collapse int...       1
7609  @aria_ahrary @TheTawniest The out of control w...       1
7610  M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...       1
7611  Police investigating after an e-bike collided ...       1
7612  The Latest: More Homes Razed by Northern Calif...       1

[7613 rows x 2 columns]>

In [8]:
#example of a negative tweet aka no natural disasters
train_data[train_data["target"] == 0]["text"].values[0]

"What's up man?"

In [9]:
#example of a negative tweet aka yes natural disasters
train_data[train_data["target"] == 1]["text"].values[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

preprocessing! 


In [10]:
def preprocess_text(text):
    # remove links and mentions
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\S+', '', text)
    # remove unwanted characters
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    # convert to lowercase
    text = text.lower()
    return text
train_data_cleaned["text"] = train_data_cleaned["text"].apply(preprocess_text)
train_data_cleaned.head()

,text,target
0,our deeds are the reason of this earthquake ma...,1
1,forest fire near la ronge sask canada,1
2,all residents asked to shelter in place are be...,1
3,13 000 people receive wildfires evacuation ord...,1
4,just got sent this photo from ruby alaska as s...,1


In [11]:
train_data_cleaned["text"] = train_data_cleaned["text"].apply(preprocess_text)

In [12]:
train_data_cleaned

,text,target
0,our deeds are the reason of this earthquake ma...,1
1,forest fire near la ronge sask canada,1
2,all residents asked to shelter in place are be...,1
3,13 000 people receive wildfires evacuation ord...,1
4,just got sent this photo from ruby alaska as s...,1
...,...,...
7608,two giant cranes holding a bridge collapse int...,1
7609,the out of control wild fires in california e...,1
7610,m1 94 01 04 utc 5km s of volcano hawaii,1
7611,police investigating after an e bike collided ...,1


In [13]:
train_data_cleaned["text"]

0       our deeds are the reason of this earthquake ma...
1                   forest fire near la ronge sask canada
2       all residents asked to shelter in place are be...
3       13 000 people receive wildfires evacuation ord...
4       just got sent this photo from ruby alaska as s...
                              ...                        
7608    two giant cranes holding a bridge collapse int...
7609     the out of control wild fires in california e...
7610             m1 94 01 04 utc 5km s of volcano hawaii 
7611    police investigating after an e bike collided ...
7612    the latest more homes razed by northern califo...
Name: text, Length: 7613, dtype: object

In [14]:
train_data_cleaned.head()

,text,target
0,our deeds are the reason of this earthquake ma...,1
1,forest fire near la ronge sask canada,1
2,all residents asked to shelter in place are be...,1
3,13 000 people receive wildfires evacuation ord...,1
4,just got sent this photo from ruby alaska as s...,1


In [15]:
type ( train_data_cleaned["text"][0][0])

str

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
X_train= vectorizer.fit_transform (train_data_cleaned["text"])
y_train = train_data_cleaned['target']
print(X_train)



  (0, 896)	0.2081243239053924
  (0, 13765)	0.24482059129521053
  (0, 5306)	0.4188071600884021
  (0, 897)	0.36176168876070036
  (0, 8205)	0.2592612160012952
  (0, 4331)	0.29049695279311155
  (0, 13063)	0.18004871319092697
  (0, 9231)	0.11992295981037408
  (0, 10636)	0.3266079333156953
  (0, 12998)	0.10165102504908612
  (0, 1161)	0.18928458735777504
  (0, 3662)	0.4188071600884021
  (0, 9445)	0.2526628358380839
  (1, 2395)	0.38504590858739146
  (1, 11342)	0.4877033840597921
  (1, 11120)	0.4877033840597921
  (1, 7447)	0.3588561292159727
  (1, 8885)	0.312861822823877
  (1, 5122)	0.23647248619734926
  (1, 5297)	0.30569754373965585
  (2, 4795)	0.21674309724049953
  (2, 9397)	0.22544536558436154
  (2, 9381)	0.14034080967621868
  (2, 4701)	0.18227566307612977
  (2, 9433)	0.18902566680833688
  :	:
  (7611, 1803)	0.5348267249240581
  (7611, 9042)	0.23435333925990826
  (7611, 13089)	0.25232202810230503
  (7611, 6911)	0.23006578323220223
  (7611, 772)	0.14079092720587183
  (7611, 11571)	0.234353339

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

In [21]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state = 42)
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [23]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'acc:{acc}')

acc:0.8102429415627052


In [21]:
df_submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
df_submission.shape

(3263, 2)

In [22]:
y_test_pred = model.predict(x_test)

In [23]:
df_submission['target'] = y_test_pred


In [24]:
df_submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,0
3260,10868,1
3261,10874,1


In [25]:
df_submission.to_csv('my_submission.csv',index=False)

Naive bayes


In [25]:
from sklearn.naive_bayes import GaussianNB
clf=GaussianNB()

In [26]:
X_train_dense=X_train.todense()
clf.fit(X_train_dense,y_train)
X_test_dense=X_test.todense()
y_pred_nb = clf.predict(X_test_dense)
acc = accuracy_score(y_test, y_pred_nb)
print(f'acc:{acc}')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


acc:0.6231122783978988


In [28]:
print(X_train)

  (0, 3289)	0.3319603317198696
  (0, 12667)	0.31740065590756417
  (0, 6279)	0.31740065590756417
  (0, 10729)	0.3070704015203169
  (0, 159)	0.2869753938640378
  (0, 995)	0.29905763370929545
  (0, 6361)	0.31740065590756417
  (0, 1981)	0.19007678502857986
  (0, 1316)	0.21104552384039066
  (0, 8427)	0.214282721616754
  (0, 7048)	0.2572905411212114
  (0, 8895)	0.2042928354543582
  (0, 13778)	0.23022366690788323
  (0, 1003)	0.09974235208551381
  (0, 6675)	0.08787599613018895
  (0, 13191)	0.08964302183087652
  (0, 9231)	0.09088580552002667
  (1, 2057)	0.3914010032130786
  (1, 2494)	0.37423427823160565
  (1, 5744)	0.3526067625186118
  (1, 14380)	0.3914010032130786
  (1, 4560)	0.2678673952279501
  (1, 11636)	0.3277208430587797
  (1, 5169)	0.24883524283396885
  (1, 1649)	0.34488756804025267
  :	:
  (6088, 772)	0.17323250397706041
  (6088, 2506)	0.26874086779326223
  (6088, 11376)	0.21663608836958242
  (6088, 10066)	0.25447808066491806
  (6088, 9475)	0.18594807779541633
  (6088, 6335)	0.218517760